# Customer Feedback Analysis

This notebook demonstrates how to analyze large-scale customer feedback using openaivec for sentiment analysis, categorization, and priority assignment.

In [1]:
import pandas as pd
from openaivec import pandas_ext
from pydantic import BaseModel
from typing import Literal

# Setup
pandas_ext.responses_model("gpt-4o-mini")

## Sample Customer Feedback Data

Let's create realistic customer feedback data representing various scenarios.

In [2]:
# Sample customer feedback data
feedback_data = [
    "The product quality is excellent, but delivery took 3 weeks. Very disappointed with shipping.",
    "Amazing customer service! Sarah helped me resolve my issue in 5 minutes. Will definitely buy again.",
    "App keeps crashing when I try to checkout. This is the third time this week. Fix your software!",
    "Love the new design updates. Much more user-friendly than before. Great job team!",
    "Pricing is too high compared to competitors. $50 for basic features is unreasonable.",
    "Website is slow and confusing. Took me 20 minutes to find what I was looking for.",
    "Perfect packaging, fast delivery, excellent product. 5 stars!",
    "Customer support never responds to emails. Been waiting 2 weeks for a refund.",
    "The mobile app crashes every time I try to upload photos. Please fix this bug.",
    "Great value for money. Been using this service for 2 years now, consistently good."
]

customer_df = pd.DataFrame({
    "customer_id": [f"CUST_{i:03d}" for i in range(1, len(feedback_data) + 1)],
    "feedback": feedback_data,
    "date": pd.date_range("2024-01-01", periods=len(feedback_data), freq="D")
})

customer_df.head()

,customer_id,feedback,date
0,CUST_001,"The product quality is excellent, but delivery...",2024-01-01
1,CUST_002,Amazing customer service! Sarah helped me reso...,2024-01-02
2,CUST_003,App keeps crashing when I try to checkout. Thi...,2024-01-03
3,CUST_004,Love the new design updates. Much more user-fr...,2024-01-04
4,CUST_005,Pricing is too high compared to competitors. $...,2024-01-05


## Structured Analysis

Define a structured output format for comprehensive feedback analysis.

In [3]:
class FeedbackAnalysis(BaseModel):
    sentiment: Literal["positive", "negative", "neutral"]
    category: Literal["product", "service", "delivery", "pricing", "technical", "website"]
    priority: Literal["low", "medium", "high", "urgent"]
    main_issue: str
    action_needed: str
    customer_emotion: Literal["satisfied", "frustrated", "angry", "delighted", "neutral"]

## AI-Powered Analysis

Process all feedback with structured analysis in a single operation.

In [4]:
# Comprehensive feedback analysis
analyzed_df = customer_df.assign(
    analysis=lambda df: df.feedback.ai.responses(
        instructions="""
        Analyze the customer feedback and provide structured insights including:
        - Sentiment classification
        - Issue categorization  
        - Priority level for follow-up
        - Main issue summary
        - Recommended action
        - Customer emotional state
        """,
        response_format=FeedbackAnalysis
    )
).ai.extract("analysis")

analyzed_df.head()

,customer_id,feedback,date,analysis_sentiment,analysis_category,analysis_priority,analysis_main_issue,analysis_action_needed,analysis_customer_emotion
0,CUST_001,"The product quality is excellent, but delivery...",2024-01-01,negative,delivery,high,Delivery took too long despite good product qu...,Investigate shipping delays and improve delive...,frustrated
1,CUST_002,Amazing customer service! Sarah helped me reso...,2024-01-02,positive,service,low,Excellent customer service experience.,Acknowledge and commend Sarah for her assistance.,delighted
2,CUST_003,App keeps crashing when I try to checkout. Thi...,2024-01-03,negative,technical,urgent,App crashes during checkout.,Fix the app crashing issue as a priority.,angry
3,CUST_004,Love the new design updates. Much more user-fr...,2024-01-04,positive,product,low,Positive feedback on new design updates.,Continue to enhance user experience based on f...,satisfied
4,CUST_005,Pricing is too high compared to competitors. $...,2024-01-05,negative,pricing,medium,Pricing perceived as too high for basic features.,Review pricing strategy in comparison to compe...,frustrated


## Business Intelligence Dashboard

Generate insights for business stakeholders.

In [5]:
# Sentiment distribution
sentiment_summary = analyzed_df.analysis_sentiment.value_counts()
print("Sentiment Distribution:")
print(sentiment_summary)

print("\n" + "="*50 + "\n")

# Priority breakdown
priority_summary = analyzed_df.analysis_priority.value_counts()
print("Priority Breakdown:")
print(priority_summary)

print("\n" + "="*50 + "\n")

# Category insights
category_summary = analyzed_df.analysis_category.value_counts()
print("Issue Categories:")
print(category_summary)

Sentiment Distribution:
analysis_sentiment
negative    6
positive    4
Name: count, dtype: int64


Priority Breakdown:
analysis_priority
low       4
high      2
urgent    2
medium    2
Name: count, dtype: int64


Issue Categories:
analysis_category
service      3
technical    2
product      2
delivery     1
pricing      1
website      1
Name: count, dtype: int64


## Action Items Generation

Create prioritized action items for different teams.

In [6]:
# High priority items
urgent_issues = analyzed_df[
    analyzed_df.analysis_priority.isin(["high", "urgent"])
].sort_values("analysis_priority")

print("🚨 URGENT ACTION ITEMS:")
for _, row in urgent_issues.iterrows():
    print(f"\n📋 Customer: {row.customer_id}")
    print(f"   Category: {row.analysis_category.upper()}")
    print(f"   Issue: {row.analysis_main_issue}")
    print(f"   Action: {row.analysis_action_needed}")

🚨 URGENT ACTION ITEMS:

📋 Customer: CUST_001
   Category: DELIVERY
   Issue: Delivery took too long despite good product quality.
   Action: Investigate shipping delays and improve delivery times.

📋 Customer: CUST_008
   Category: SERVICE
   Issue: Customer support unresponsive regarding refund.
   Action: Address customer support responsiveness and expedite refund process.

📋 Customer: CUST_003
   Category: TECHNICAL
   Issue: App crashes during checkout.
   Action: Fix the app crashing issue as a priority.

📋 Customer: CUST_009
   Category: TECHNICAL
   Issue: Mobile app crashes when uploading photos.
   Action: Fix the app crashing issue as a priority.


## Conclusion

This example shows how openaivec can transform raw customer feedback into:
- Structured business intelligence
- Prioritized action items
- Team-specific insights
- Automated categorization

Scale this approach to thousands of reviews for comprehensive customer insight analysis.